# Proceso de Curación

Se pretende construir un modelo predictivo de la calidad del aire en las principales ciudades de España a partir de la meteorología del lugar para tratar de determinar el efecto del confinamiento y las restricciones debidas a la pandemia del COVID-19.

In [1]:
suppressMessages(library(tidyverse))

setwd("~/Repositories/AirQualityCOVID/")

Para ello se han de obtener datos tanto de calidad del aire como meteorológicos suficientes para contruir del modelo predictivo. Se ha decidido utilizar el periodo de tiempo desde el `1-Enero-2013` al `31-Diciembre-2019` para entrenar el modelo.

Se ha escogido tomar datos desde el $2013$ para la creación del modelo debido a que es el año a partir del cual hay datos en el portal de la [_European Enviroment Aency_ (**EEA**)](https://discomap.eea.europa.eu/map/fme/AirQualityExport.htm)

In [2]:
suppressMessages(library(lubridate))

start_dt <- ymd("2013-01-01")
end_dt <- ymd("2020-12-30")

---

## Calidad del Aire

> _El proceso de curación completo de los datos de calidad del aire se ha realizado mediante el script `src/Curation/airQuality.R`._

Las restricciones de movilidad debidas al confinamiento por el COVID-19 deberían haber supuesto una disminución en el uso de vehículos, lo cual repercutiría directamente en los niveles de concentración de los contaminante producidos por los vehículos. 

Por ello en este estudio nos hemos centrado en las estaciones de calidad del aire de tipo **tráfico urbano** situadas en ciudades españolas con más de $> 100 000$ habitantes.

Los contaminantes escogidos han sido: 
* _Óxido de Nitrógeno_ ($NO$)
* _Dióxido de Nitrógeno_ ($NO_2$)
* _Ozono_ ($O_3$)
* _Partículas de Materia de diámetro $\leq 10 \mu m$_ ($PM10$)
* _Partículas de Materia de diámetro $\leq 2.5 \mu m$_ ($PM2.5$).

In [3]:
site_type <- "traffic"
site_area <- "urban"

pollutants <- c("no", "no2", "o3", "pm10", "pm2.5")

En el archivo `data/xlsx/estaciones-CA-JA.xlsx` se encuentra información sobre las estaciones de calidad del aire de las ciudades españolas de $> 100 000$ habitantes.

In [4]:
suppressMessages(library(openxlsx))

# AQ station in cities with more than 100000 inhabitants
sites.100mil <- read.xlsx("data/xlsx/estaciones-CA-JA.xlsx",
                          sheet="ciudades-100000-A") %>% 
                    select("Municipio", "Población",
                           "Estación.tráfico", "Código.estación") 

Para la obtención de los datos de calidad del aire se ha utilizado el paquete [`saqgetr`](https://github.com/skgrange/saqgetr), el cual utiliza como principal fuente de datos la **EEA**.

Por ello se utiliza dicho paquete para comprobar que la información del archivo anterior concuerda con la disponible por la **EEA** y las estaciones cumplen con los requisitos mínimos descritos anteriormente.

In [8]:
suppressMessages(library(saqgetr))

spain.sites <- get_saq_sites() %>%
    filter(country == "spain",
           site %in% sites.100mil$"Código.estación",
           site_type == site_type,
           site_area == site_area,
           date_start <= start_dt,
           ) %>%
    select(site, site_name, latitude, longitude, elevation, 
           country, site_type, site_area, date_start, date_end)

sites.AQ <- merge(x = spain.sites,
                  y = sites.100mil,
                  by.x = "site", by.y="Código.estación",
                  all.x = TRUE) 
head(sites.AQ)

,site,site_name,latitude,longitude,elevation,country,site_type,site_area,date_start,date_end,Municipio,Población,Estación.tráfico
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dttm>,<dttm>,<chr>,<dbl>,<chr>
1,es0041a,DIRECCIÓN DE SALUD,43.25883,-2.945650,32,spain,traffic,urban,1986-04-01,2021-04-10 23:00:00,Bilbao,346843,María Diaz de Haro
2,es0110a,ERANDIO,43.30268,-2.977240,4,spain,traffic,urban,1997-01-01,2021-04-10 23:00:00,Bilbao,24350,Erandio
3,es0115a,PLAZA DE ESPAÑA,40.42417,-3.712222,637,spain,traffic,urban,1986-01-01,2021-04-10 23:00:00,Madrid,3266127,PLAZA DE ESPAÑA
4,es0118a,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19,2021-04-10 23:00:00,Madrid,3266128,ESCUELAS AGUIRRE
5,es0120a,RAMÓN Y CAJAL,40.45167,-3.677222,708,spain,traffic,urban,2002-01-01,2021-04-10 23:00:00,Madrid,3266129,RAMÓN Y CAJAL
6,es0817a,LA RANILLA,37.38425,-5.959620,29,spain,traffic,urban,1997-01-01,2021-04-11 00:00:00,Sevilla,688593,LA RANILLA


Sin embargo, esta información solo hace referencia a los datos generales de la estación, sin indicar si existen datos suficientes para el estudio de cada contaminante. Esto se debe comprobar de forma individual para cada contaminante una vez descargado los datos.

```R
    data.AQ <- get_saq_observations(
        site = "es0000a"
        variable = "XO",
        start = start_dt,
        end = end_dt,
        valid_only = TRUE,
        verbose = FALSE
    )
```

Para cada contaminante de estudio de cada estación se obtienen los siguientes valores:

* **start_yr**: Fecha de comienzo de toma de datos. El valor mínimo posible es `start_dt`.
* **end_yr**: Fecha de fin de toma de datos. El valor máximo posible es `end_dt`.
* **hv.min**: `Booleano` si hay datos con resolución diaria suficientes ($> 80\%$) durante el periodo de mayor interes del estudio.
    > Se toma el intervalo de tiempo `1-Marzo-2020` al `30-Junio-2020` como el periodo de mayor interes del estudio ya que comprende los meses desde el comienzo del primer estado de alarma hasta la vuelta a la primera "normalidad" (fin de las fases de la desescalada) 
* **mss.wk**: Número de valores nulos con resolución semanal. Número de semanas faltantes.
* **mss.mnth**: Número de valores nulos con resolución mensual. Número de meses faltantes.
* **mss.yr**: Número de valores nulos con resolución anual. Número de años faltantes.

A partir de estos valores se han seleccionado aquellos contaminantes en cada estación con:
```R
    hv.min == TRUE
    mss.yr < 5
```

In [6]:
valid.df <- read.csv("data/Curation/checked_AQ.csv")
head(valid.df)

,site,Pollutant,site_name,latitude,longitude,elevation,country,site_type,site_area,date_start,date_end,Municipio,Población,Estación.tráfico
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,es0041a,no2,DIRECCIÓN DE SALUD,43.25883,-2.94565,32,spain,traffic,urban,1986-04-01 00:00:00,2021-04-10 23:00:00,Bilbao,346843,María Diaz de Haro
2,es0041a,pm10,DIRECCIÓN DE SALUD,43.25883,-2.94565,32,spain,traffic,urban,1986-04-01 00:00:00,2021-04-10 23:00:00,Bilbao,346843,María Diaz de Haro
3,es0110a,no,ERANDIO,43.30268,-2.97724,4,spain,traffic,urban,1997-01-01 00:00:00,2021-04-10 23:00:00,Bilbao,24350,Erandio
4,es0110a,no2,ERANDIO,43.30268,-2.97724,4,spain,traffic,urban,1997-01-01 00:00:00,2021-04-10 23:00:00,Bilbao,24350,Erandio
5,es0110a,pm10,ERANDIO,43.30268,-2.97724,4,spain,traffic,urban,1997-01-01 00:00:00,2021-04-10 23:00:00,Bilbao,24350,Erandio
6,es0110a,pm2.5,ERANDIO,43.30268,-2.97724,4,spain,traffic,urban,1997-01-01 00:00:00,2021-04-10 23:00:00,Bilbao,24350,Erandio


---

## Meteorológicos

### Fuente: AEMET

### Fuente: NOAA

### Fuente: ERA5-Land